In [1]:
from __future__ import absolute_import, division, print_function
import pandas as pd
from six.moves import urllib
import shutil
import tensorflow as tf


/Users/i550012/virtualenv/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/i550012/virtualenv/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/i550012/virtualenv/tensorflow1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/i550012/virtualenv/tensorflow1

In [2]:
TRAIN_FILE_NAME = "census/adult.data"
TEST_FILE_NAME = "census/adult.test"
urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", TRAIN_FILE_NAME)
urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", TEST_FILE_NAME)


('census/adult.test', <http.client.HTTPMessage at 0x1253905d0>)

In [3]:
CSV_COLUMNS = [
    "age", "workclass", "fnlwgt",  "education", "education_num", "marital_status",
    "occupation", "relationship",  "race", "gender", "capital_gain", "capital_loss",
    "hours_per_week", "native_country", "income_bracket"
]


In [4]:
df = pd.read_csv(TRAIN_FILE_NAME, names= CSV_COLUMNS, skipinitialspace=True, skiprows=1)

In [5]:
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [6]:

TRIMMED_REORDERED_COLUMNS = [
    "age", "workclass", "education", "education_num", "marital_status",
    "relationship",  "race", "gender", "occupation",
    "hours_per_week", "native_country", "income_bracket"
]


In [7]:
df = df[TRIMMED_REORDERED_COLUMNS]
df.head()

,age,workclass,education,education_num,marital_status,relationship,race,gender,occupation,hours_per_week,native_country,income_bracket
0,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Husband,White,Male,Exec-managerial,13,United-States,<=50K
1,38,Private,HS-grad,9,Divorced,Not-in-family,White,Male,Handlers-cleaners,40,United-States,<=50K
2,53,Private,11th,7,Married-civ-spouse,Husband,Black,Male,Handlers-cleaners,40,United-States,<=50K
3,28,Private,Bachelors,13,Married-civ-spouse,Wife,Black,Female,Prof-specialty,40,Cuba,<=50K
4,37,Private,Masters,14,Married-civ-spouse,Wife,White,Female,Exec-managerial,40,United-States,<=50K


In [8]:
df['gender'].unique()

array(['Male', 'Female'], dtype=object)

In [9]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", df["gender"].unique())
race = tf.feature_column.categorical_column_with_vocabulary_list("race", df["race"].unique())
education = tf.feature_column.categorical_column_with_vocabulary_list("education", df["education"].unique())
marital_status = tf.feature_column.categorical_column_with_vocabulary_list("marital_status", df["marital_status"].unique())
relationship = tf.feature_column.categorical_column_with_vocabulary_list("relationship", df["relationship"].unique())
workclass= tf.feature_column.categorical_column_with_vocabulary_list("workclass", df["workclass"].unique())



In [10]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")


In [11]:
age_buckets = tf.feature_column.bucketized_column(age, boundaries=[18,25,30,35,40,45,50,55,60,65])

In [12]:
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

In [13]:
base_columns = [
    gender, race, marital_status, workclass, occupation, native_country, age_buckets, education
]

In [15]:
crossed_columns = [
    tf.feature_column.crossed_column(["education", "occupation"], hash_bucket_size=1000),
    tf.feature_column.crossed_column([age_buckets, "education", "occupation"], hash_bucket_size=1000),
    tf.feature_column.crossed_column(["native_country", "occupation"], hash_bucket_size=1000)
]

In [16]:
deep_columns = [ education_num, hours_per_week]

In [17]:
def input_fn(file_name, num_epochs, shuffle): 
    df = pd.read_csv(file_name, names=CSV_COLUMNS, skipinitialspace=True, skiprows=1)
    df = df[TRIMMED_REORDERED_COLUMNS]
    df = df.dropna(how="any", axis=0)
    labels = df["income_bracket"].apply(lambda x: ">50K" in x).astype(int)
    return tf.estimator.inputs.pandas_input_fn(x=df, y= labels, batch_size=100, num_epochs=num_epochs, shuffle=shuffle, num_threads=5)

In [30]:
MODEL_DIR = "./linear_classifier"
import os
os.makedirs(MODEL_DIR)
shutil.rmtree(MODEL_DIR)
linear_estimator = tf.estimator.LinearClassifier(model_dir=MODEL_DIR, feature_columns=base_columns+crossed_columns+deep_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './linear_classifier', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x125f79150>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [31]:
linear_estimator.train(input_fn=input_fn(TRAIN_FILE_NAME, num_epochs=None, shuffle=True), steps=1000)




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2024-04-04 09:38:23.701386: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA


Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into ./linear_classifier/model.ckpt.
INFO:tensorflow:loss = 69.31472, step = 1
INFO:tensorflow:global_step/sec: 141.494
INFO:tensorflow:loss = 33.72049, step = 101 (0.707 sec)
INFO:tensorflow:global_step/sec: 314.513
INFO:tensorflow:loss = 39.91265, step = 201 (0.318 sec)
INFO:tensorflow:global_step/sec: 295.939
INFO:tensorflow:loss = 32.669117, step = 301 (0.338 sec)
INFO:tensorflow:global_step/sec: 301.039
INFO:tensorflow:loss = 29.18893, step = 401 (0.332 sec)
INFO:tensorflow:global_step/sec: 309.004
INFO:tensorflow:loss = 36.321507, step = 501 (0.324 sec)
INFO:tensorflow:global_step/sec: 278.156
INFO:tensorflow:loss = 34.26846, step = 601 (0.359 sec)
INFO:tensorflow:global_step/sec: 288.825
INFO:tensorflow:loss = 25.266666, step = 701 (0.347 sec)
INFO:tensorflow:global_step/sec: 262.176
INFO:tensorflow:loss = 35.13993, step = 801 (0.380 sec)
INFO:tensorflow:gl

In [32]:
results = linear_estimator.evaluate(input_fn=input_fn(TEST_FILE_NAME, num_epochs=1, shuffle=False), steps=None)


INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2024-04-04T09:39:48Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./linear_classifier/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2024-04-04-09:39:55
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.8386671, accuracy_baseline = 0.75918305, auc = 0.8960385, auc_precision_recall = 0.725801, average_loss = 0.3428352, global_step = 1000, label/mean = 0.24081695, loss = 34.283516, precision = 0.7615198, prediction/mean = 0.20802665, recall = 0.48055094
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: ./linear_classifier/model.ckpt-1000


In [33]:
for key in sorted(results): 
    print ("%s: %s"%(key, results[key]))
    

accuracy: 0.8386671
accuracy_baseline: 0.75918305
auc: 0.8960385
auc_precision_recall: 0.725801
average_loss: 0.3428352
global_step: 1000
label/mean: 0.24081695
loss: 34.283516
precision: 0.7615198
prediction/mean: 0.20802665
recall: 0.48055094
